**Prepare el Dataset**

In [1]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("<subcription_id>","<resource_group>","<workspace>")

Found the config file in: /config.json


In [2]:
dataset_parent_dir = "./data"
dataset_name = "Car-Bike-Dataset"
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")


In [3]:
training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")


**Sent to Training**

In [4]:
my_training_data_input = Input(type=AssetTypes.MLTABLE,path=training_mltable_path)
my_validation_data_input = Input(type=AssetTypes.MLTABLE,path=validation_mltable_path)

In [5]:
compute_name = "gpl-cluster-nc6sv3"
def create_cluster_computer(compute_name):
    try:
        _ = ml_client.compute.get(compute_name)
        print("Found Existing compute target")
    except:
        print("Creating a new Computer target")
        conpute_config = AmlCompute(
            name=compute_name,
            type="amlcompute",
            size="Standard_NC6s_v3",
            idle_time_before_scale_down=120,
            min_instances=0,
            max_instances=4
        )
        ml_client.begin_create_or_update(conpute_config).result()
        print("Done")

create_cluster_computer(compute_name)

Creating a new Computer target
Done


Create image classification job

In [6]:
exp_name = "dl-car-bike-objects"

image_classfication_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name='label',
    primary_metric="accuracy",
    tags={"exp-name":"Job for image classification with car bikes object ds"}
)

image_classfication_job.set_limits(max_trials=4, max_concurrent_trials=2)

returned_job = ml_client.jobs.create_or_update(image_classfication_job)

Uploading training-mltable-folder (1.05 MBs): 100%|██████████| 1049489/1049489 [00:00<00:00, 21392837.96it/s]


Uploading validation-mltable-folder (0.26 MBs): 100%|██████████| 262810/262810 [00:00<00:00, 6401306.82it/s]




In [9]:
import mlflow

MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/test-ml-rc-101


In [10]:
from mlflow.tracking.client import MlflowClient

mlflow_client = MlflowClient()

In [11]:

job_name = returned_job.name

mlflow_parent_run =  mlflow_client.get_run(job_name)
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_binary': 1.0,
 'AUC_macro': 1.0,
 'AUC_micro': 1.0,
 'AUC_weighted': 1.0,
 'accuracy': 0.99875,
 'accuracy_train': 0.99094,
 'average_precision_score_binary': 1.0,
 'average_precision_score_macro': 1.0,
 'average_precision_score_micro': 1.0,
 'average_precision_score_weighted': 1.0,
 'f1_score_binary': 0.99875,
 'f1_score_macro': 0.99875,
 'f1_score_micro': 0.99875,
 'f1_score_weighted': 0.99875,
 'log_loss': 0.00238,
 'log_loss_train': 0.02392,
 'precision_score_binary': 0.99751,
 'precision_score_macro': 0.99875,
 'precision_score_micro': 0.99875,
 'precision_score_weighted': 0.99875,
 'recall_score_binary': 1.0,
 'recall_score_macro': 0.99875,
 'recall_score_micro': 0.99875,
 'recall_score_weighted': 0.99875}, params={}, tags={'automl_best_child_run_id': 'amusing_camel_ssmp0rm8pv_HD_0',
 'exp-name': 'Job for image classification with car bikes object ds',
 'fit_time_000': 'NaN',
 'is_gpu': 'True',
 'iteration_000': '0',
 'mlflow.rootRunId': 'amusin

In [12]:
print(mlflow_parent_run.data.tags['automl_best_child_run_id'])

amusing_camel_ssmp0rm8pv_HD_0


In [13]:
best_child_run_id = mlflow_parent_run.data.tags['automl_best_child_run_id']
best_run = mlflow_client.get_run(best_child_run_id)

**Metrics Best Model**

In [14]:
best_run.data.metrics

{'log_loss': 0.00238,
 'accuracy_train': 0.99094,
 'log_loss_train': 0.02392,
 'precision_score_weighted': 0.99875,
 'f1_score_binary': 0.99875,
 'recall_score_binary': 1.0,
 'average_precision_score_macro': 1.0,
 'recall_score_macro': 0.99875,
 'f1_score_macro': 0.99875,
 'precision_score_macro': 0.99875,
 'f1_score_weighted': 0.99875,
 'average_precision_score_micro': 1.0,
 'accuracy': 0.99875,
 'precision_score_binary': 0.99751,
 'recall_score_weighted': 0.99875,
 'AUC_weighted': 1.0,
 'AUC_micro': 1.0,
 'recall_score_micro': 0.99875,
 'AUC_macro': 1.0,
 'AUC_binary': 1.0,
 'precision_score_micro': 0.99875,
 'f1_score_micro': 0.99875,
 'average_precision_score_binary': 1.0,
 'average_precision_score_weighted': 1.0}

**descargar el modelo un folder local**

In [15]:
import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [16]:
local_path = mlflow_client.download_artifacts(best_run.info.run_id, "outputs", local_dir)
local_path

2023/10/22 16:48:41 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/testrc101/code/Users/ron.todo.poderoso/proyecto/artifact_downloads/outputs'

**Crear el endpoint**

In [17]:
from azure.ai.ml.entities import (ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment,CodeConfiguration,ProbeSettings)


In [30]:
import datetime
online_endpoint_name = "car-bike-items" + datetime.datetime.now().strftime("%m%d%H%M")

endpoint = ManagedOnlineEndpoint(
    name = online_endpoint_name,
    description="this is a sample of a endpoint",
    auth_mode="key",
    tags={"foo":"bar"}
)

In [31]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://car-bike-items10221709.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://car-bike-items10221709.westus.inference.ml.azure.com/swagger.json', 'name': 'car-bike-items10221709', 'description': 'this is a sample of a endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourcegroups/rchoque/providers/microsoft.machinelearningservices/workspaces/test-ml-rc-101/onlineendpoints/car-bike-items10221709', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:68770f40-7934-479c-9806-116f9382554b:67c994e9-8988-4585-a23e-9ed7e8b2a0b7?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGro

**Registrar el Modelo**


In [32]:
best_run.info.run_id

'amusing_camel_ssmp0rm8pv_HD_0'

In [33]:
model_name = "car-bike-object-detection-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description = "my sample image",
    type= AssetTypes.MLFLOW_MODEL,
)
model

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': True, 'auto_delete_setting': None, 'name': 'car-bike-object-detection-model', 'description': 'my sample image', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/testrc101/code/Users/ron.todo.poderoso/proyecto', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fee57c10d90>, 'version': None, 'latest_version': None, 'path': 'azureml://jobs/amusing_camel_ssmp0rm8pv_HD_0/outputs/artifacts/outputs/mlflow-model/', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model', 'stage': None})

In [34]:
registered_model = ml_client.models.create_or_update(model)

In [35]:
registered_model.id

'/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/test-ml-rc-101/models/car-bike-object-detection-model/versions/2'

**Deployment**


In [38]:
from azure.ai.ml.entities import OnlineRequestSettings
req_timeout = OnlineRequestSettings(request_timeout_ms=90000)

deployment = ManagedOnlineDeployment(
    name = "id-mc-car-bike-items-mlflow-depl",
    endpoint_name  = online_endpoint_name,
    model = registered_model.id,
    #gpu
    instance_type = "Standard_E4s_v3",
    instance_count = 1,
    request_settings = req_timeout
)

In [37]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint car-bike-items10221709 exists


.........................

HttpResponseError: (None) OutOfQuota: Not enough cluster CPU quota. The amount of additional CPU quota requested is 8, and you are currently using 14. Your amount of maximum quota is 20. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
Code: None
Message: OutOfQuota: Not enough cluster CPU quota. The amount of additional CPU quota requested is 8, and you are currently using 14. Your amount of maximum quota is 20. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
Exception Details:	(None) OutOfQuota: Not enough cluster CPU quota. The amount of additional CPU quota requested is 8, and you are currently using 14. Your amount of maximum quota is 20. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
	Code: None
	Message: OutOfQuota: Not enough cluster CPU quota. The amount of additional CPU quota requested is 8, and you are currently using 14. Your amount of maximum quota is

**Inferencias / Predicciones**

In [39]:
import base64

online_endpoint_name = "car-bike-items10211955"
deployment_name = "car-bike-object-detection-mode-2"

sample_image =os.path.join(dataset_dir,"Bike","Bike (10).jpg")

def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()
        
request_json = {
    "input_data":{
        "columns":['image'],
        "data":[base64.encodebytes(read_image(sample_image)).decode("utf-8")]
    }
}

In [40]:
request_file_name = "sample_request_data.json"

with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [41]:
resp = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file=request_file_name
)

resp

ResourceNotFoundError: (ParentResourceNotFound) Failed to perform 'read' on resource(s) of type 'workspaces/onlineEndpoints/deployments', because the parent resource '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/test-ml-rc-101/onlineEndpoints/car-bike-items10211955' could not be found.
Code: ParentResourceNotFound
Message: Failed to perform 'read' on resource(s) of type 'workspaces/onlineEndpoints/deployments', because the parent resource '/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/test-ml-rc-101/onlineEndpoints/car-bike-items10211955' could not be found.